In [1]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

DEVICE = "cuda"


import torch, random, numpy as np
from transformers import set_seed

def set_all_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    set_seed(seed)
    torch.backends.cudnn.deterministic = True

set_all_seeds(9)

from safetensors import safe_open

In [58]:
# rust
# cargo run -p smol_vlm --features cuda -- -i .vscode/angela-porter-2021-jan-25.jpg -p "Can you describe the image?" --sample-length 500
# cargo run -p smol_vlm --features cuda -- -p "A real-valued function f defined on the real line is called an even function if f(-t) = f(t) for each real number t. Prove that the set of even functions defined on the real line with the operations of addition and scalar multiplication defined in Example 3 is a vector space." --sample-length 50
# cargo run -p smol_vlm --features cuda -- -p "Can you describe the image?" --sample-length 500
# cargo run -p smol_vlm --features cuda -- -p "What is life?" --sample-length 500


# image1 = load_image("https://media.istockphoto.com/id/485371557/photo/twilight-at-spirit-island.jpg?s=612x612&w=0&k=20&c=FSGliJ4EKFP70Yjpzso0HfRR4WwflC6GKfl4F3Hj7fk=")
# image2 = load_image("https://huggingface.co/spaces/merve/chameleon-7b/resolve/main/bee.jpg")
# image1 = Image.open("../../../.vscode/fuji-mountain-in-autumn.jpg")
image1 = load_image("https://artwyrd.com/wp-content/uploads/2021/01/angela-porter-2021-jan-25.jpg")
print(f"Image 1 size: {image1.size}")


processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
messages = [
    {
        "role": "user",
        "content": [
            # {"type": "image"},
            {"type": "text", "text": "Can you describe the image?"}
            # {"type": "text", "text": "What is life?"}
            # {"type": "text", "text": "A real-valued function f defined on the real line is called an even function if f(-t) = f(t) for each real number t. Prove that the set of even functions defined on the real line with the operations of addition and scalar multiplication defined in Example 3 is a vector space."}
        ]
    },
]

Image 1 size: (900, 900)


In [59]:
# Initialize model directly on CUDA without Flash Attention
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-Instruct",
    torch_dtype=torch.bfloat16,
    # _attn_implementation="flash_attention_2",  # Commented out Flash Attention
    device_map="cuda",
)


embeddings = {}
counter = 0

def hook_fn(name):
    def hook(module, input, output):
        global counter
        embeddings[name+f"_{counter}"] = output.detach().cpu()
        counter += 1

    return hook

# Register hooks for different layers
model.get_input_embeddings().register_forward_hook(hook_fn("input_embeddings"))

type(model)

transformers.models.idefics3.modeling_idefics3.Idefics3ForConditionalGeneration

In [60]:
# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
# inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = processor(text=prompt, return_tensors="pt")
inputs = inputs.to("cuda")

print(inputs["input_ids"])
# Generate outputs
outputs = model.generate(
    **inputs,
    max_new_tokens=500,
    output_scores=True,           # Return logits for each generated token
    return_dict_in_generate=True, # Return detailed output object
    do_sample=False,  # Use greedy decoding (highest logit)
)
# generated_texts = processor.batch_decode(
#     generated_ids,  # Slice to include only the first token
#     skip_special_tokens=True,
# )

print(">>", outputs.sequences.shape)
print("!!", len(outputs.scores))
print(">>", outputs.sequences)
print("!!", outputs.scores[0])

print(embeddings.keys())

tensor([[    1, 11126,    42,  1978,   346,  5125,   260,  2443,    47, 49154,
           198,  9519,  9531,    42]], device='cuda:0')
>> torch.Size([1, 514])
!! 500
>> tensor([[    1, 11126,    42,  1978,   346,  5125,   260,  2443,    47, 49154,
           198,  9519,  9531,    42,   378,  2443, 21559,   253,  6621,   429,
           253, 15879,  3562,    28,  4170,   253, 15879,  5182,  6621,    30,
           378,  4054,   314,   253, 21301,    28,   837,   827, 15713, 16328,
           359,  6934,   281,  7604,    30,   378, 21301,   314,  4412,   351,
          1461,  2728,    28,  1285,    42,  1116,    33,    30,  1903, 38257,
           370,  4253,   378,  2443,  2744,   827,  1507, 16328,    28,   582,
           335,   260,  2049,   284,   260,   550,   335,   260,  1048,    30,
          3768,  6237,   314,  8168,   282,  3824,  6333,    28,   511,  9031,
           351,   896,  1995,   282,  7921,   284, 23812,    30,   378,  6333,
           359,  6934,   281,  7604,    2

In [ ]:
with safe_open("../../../.vscode/rust_output.safetensors", framework="pt", device="cpu") as f:
    # for i in range(0,400):
    for i in range(180,184):
        rust_logits = f.get_tensor(f"logits_{i}")  # Preserves bf16 dtype perfectly!
        python_logits = outputs.scores[i][0].cpu()
        err = torch.sum((rust_logits-python_logits)**2)
        mse_logits = torch.nn.functional.mse_loss(rust_logits, python_logits)
        mae_logits = torch.nn.functional.l1_loss(rust_logits, python_logits)
        # print(err)
        # print(rust_logits.shape, rust_logits.dtype, rust_logits.device)
        # print(python_logits.shape, python_logits.dtype, python_logits.device)

        rust_embeds = f.get_tensor(f"embeds_{i}")  # Preserves bf16 dtype perfectly!
        python_embeds = embeddings[f"input_embeddings_{i}"].cpu()
        mse_embeds = torch.nn.functional.mse_loss(rust_embeds, python_embeds)
        mae_embeds = torch.nn.functional.l1_loss(rust_embeds, python_embeds)
        print(f"[{i:>3}] EMBEDS(MSE:{mse_embeds.item():.8f}, MAE:{mae_embeds.item():.8f})  LOGITS(MSE:{mse_logits.item():.8f}, MAE:{mae_logits.item():.8f}) ")

        top_logits_python = python_logits.argsort(descending=True)[:15]
        top_logits_rust = rust_logits.argsort(descending=True)[:15]
        print(f"Top Python logits: {top_logits_python}, values: {[python_logits[i] for i in top_logits_python]}, token: {[processor.tokenizer.decode([i]) for i in top_logits_python]}")
        print(f"Top Rust logits: {top_logits_rust}, values: {[rust_logits[i] for i in top_logits_rust]}, token: {[processor.tokenizer.decode([i]) for i in top_logits_rust]}")

/tmp/ipykernel_110483/1939503080.py:15: UserWarning: Using a target size (torch.Size([1, 14, 2048])) that is different to the input size (torch.Size([14, 2048])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mse_embeds = torch.nn.functional.mse_loss(rust_embeds, python_embeds)
/tmp/ipykernel_110483/1939503080.py:16: UserWarning: Using a target size (torch.Size([1, 14, 2048])) that is different to the input size (torch.Size([14, 2048])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mae_embeds = torch.nn.functional.l1_loss(rust_embeds, python_embeds)
/tmp/ipykernel_110483/1939503080.py:15: UserWarning: Using a target size (torch.Size([1, 1, 2048])) that is different to the input size (torch.Size([1, 2048])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mse_embeds = torch.nn.functional.mse_loss(rust_embeds, pyth

[  0] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00368365, MAE:0.04770055) 
[  1] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00273523, MAE:0.04126414) 
[  2] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00198119, MAE:0.03487296) 
[  3] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00723317, MAE:0.07567627) 
[  4] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00104910, MAE:0.02523484) 
[  5] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00149385, MAE:0.02994826) 
[  6] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00311820, MAE:0.04541663) 
[  7] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00167503, MAE:0.03289211) 
[  8] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00216251, MAE:0.03645422) 
[  9] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00183537, MAE:0.03345570) 
[ 10] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00256069, MAE:0.04102043) 
[ 11] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(M

In [63]:
tokenP = [    1, 11126,    42,   330,  1345,    29, 34270,  1517,   275,  4355,
           335,   260,  1345,  1761,   314,  1217,   354,   908,  1517,   585,
           275, 10242,   100,    25,   446,   275,    24,   100,    25,   327,
           971,  1345,  1230,   252,    30,  1053,   307,   338,   260,   932,
           282,   908,  3691,  4355,   335,   260,  1345,  1761,   351,   260,
          4261,   282,  1706,   284, 26727, 17385,  4355,   281, 12066,   216,
            35,   314,   253,  8431,  1898,    30, 49154,   198,  9519,  9531,
            42]
tokenR = [1, 11126, 42, 330, 1345, 29, 34270, 1517, 275, 4355, 335, 260, 1345, 1761, 314, 1217, 354, 908, 1517, 585, 275, 10242, 100, 25, 446, 275, 24, 100, 25, 327, 971, 1345, 1230, 252, 30, 1053, 307, 338, 260, 932, 282, 908, 3691, 4355, 335, 260, 1345, 1761, 351, 260, 4261, 282, 1706, 284, 26727, 17385, 4355, 281, 12066, 216, 35, 314, 253, 8431, 1898, 30, 49154, 198, 9519, 9531, 42]

print("Token same?", tokenP == tokenR)


rngDiffP1 = """The image features a detailed and intricate design, which appears to be a circular pattern. The central part of the design consists of a large, circular motif, which is composed of numerous interconnected elements. The central part is composed of a series of interconnected leaf-like shapes, each with a unique and detailed design. These leaf-like shapes are arranged in a circular pattern, creating a sense of depth and complexity.

The background of the design is filled with a dark green color, which provides a stark contrast to the lighter green and white elements of the central motif. The dark green background enhances the visual impact of the intricate patterns and designs within the central motif.

The image also includes a small, white text in the lower right corner, which reads "(c) Angela Porter | Artwyrocom | 25 Jan 2021". This text is written in a sans-serif font, which is clear and easy to read. The text provides information about the artist, the date, and the source of the image.

The overall design of the image is highly detailed and visually appealing, showcasing a blend of intricate patterns and a rich color palette. The use of a dark green background and the intricate leaf-like shapes create a sense of depth and complexity, making the image visually engaging.

In summary, the image features a circular, detailed design composed of interconnected leaf-like shapes, set against a dark green background. The central motif is composed of a series of interconnected leaf-like shapes arranged in a circular pattern, creating a sense of depth and complexity. The image is signed by the artist, Angela Porter, and dated 25th January 2021."""

rngDiffP2 = """The image features a detailed and intricate design, which appears to be a circular pattern. The central part of the design consists of a large, circular motif, which is composed of numerous interconnected elements. The central part is composed of a series of interconnected leaf-like shapes, each with a unique and detailed design. These leaf-like shapes are arranged in a circular pattern, creating a sense of depth and complexity.

The background of the design is filled with a dark green color, which provides a stark contrast to the lighter green and white elements of the central motif. The dark green background enhances the visual impact of the intricate patterns and designs within the central motif.

The image also includes a small, white text in the lower right corner, which reads "(c) Angela Porter | Artwyrocom | 25 Jan 2021". This text is written in a sans-serif font, which is clear and easy to read. The text provides information about the artist, the date, and the source of the image.

The overall design of the image is highly detailed and visually appealing, showcasing a blend of intricate patterns and a rich color palette. The use of a dark green background and the intricate leaf-like shapes create a sense of depth and complexity, making the image visually engaging.

In summary, the image features a circular, detailed design composed of interconnected leaf-like shapes, set against a dark green background. The central motif is composed of a series of interconnected leaf-like shapes arranged in a circular pattern, creating a sense of depth and complexity. The image is signed by the artist, Angela Porter, and dated 25th January 2021."""

print("Python version invariant of seed?", rngDiffP1 == rngDiffP2)


tknP = [[    1, 11126,    42,  1978,   346,  5125,   260,  2443,    47, 49154,
           198,  9519,  9531,    42,   378,  2443, 21559,   253,  6621,   429,
           253, 15879,  3562,    28,  4170,   253, 15879,  5182,  6621,    30,
           378,  4054,   314,   253, 21301,    28,   837,   827, 15713, 16328,
           359,  6934,   281,  7604,    30,   378, 21301,   314,  4412,   351,
          1461,  2728,    28,  1285,    42,  1116,    33,    30,  1903, 38257,
           370,  4253,   378,  2443,  2744,   827,  1507, 16328,    28,   582,
           335,   260,  2049,   284,   260,   550,   335,   260,  1048,    30,
          3768,  6237,   314,  8168,   282,  3824,  6333,    28,   511,  9031,
           351,   896,  1995,   282,  7921,   284, 23812,    30,   378,  6333,
           359,  6934,   281,  7604,    28,   351,   634,   282,   601, 17442,
           281, 20470,  7604,    28,   979,  1449,   359,  6934,   281,   540,
          9964, 34837,    30,  1116,    34,    30,  1903,  1882, 31146,   284,
         11889,   270,  4253,   378,  6333,   359,  9031,   351,   253,  3175,
           282,  7921,    28,  1285, 29871,    28, 45540,    28, 16418,    28,
           284, 30553,    30,   378, 23812,   502,   359,  9064,   314,   597,
          9771,    28,   351,   634,  6333,  9064, 29003,    28,  1248, 23812,
            28,   284, 30553,    30,  1116,    35,    30,  1903,    54,   403,
          7265,  6577,   756,  4253,   533,   260,  3969,    28,   665,   359,
          1545, 28557,  6734,    28,   715,   347, 26225,    28, 31721, 20273,
            28,   284,   550, 26271,  7095,    30,  1216,  6734,   359,   441,
          3319,  2773,   281,   260,  5182,   564,   803,   288,   260, 28557,
          5264,   282,   260,  6621,    30,  1116,    36,    30,  1903, 42284,
          4385,  4627,  4253,   378, 21301,   314,   253,  4749,    28,  1440,
          1557,   351,   253,  3175,   282, 12979,  3004,    28,  1285, 12610,
            28, 20232,    28,   284,  6495,    30,   378,  2118,   314, 19425,
            28,   351,   634,  1721,   282,  2375,  5372,   284,  1449,   282,
          1759,    28, 19284,  2763,    30,  1116,    37,    30,  1903, 45123,
          4253,   378,  6376,   314, 13178,   347,   253,  2437,  1194,    28,
           351,   787, 10695,   355,  5249,    30,   378,  2388,   314, 23952,
         26388,    28, 17462,   253,  3091, 14654,   690,   260, 21301,    30,
          1116,    38,    30,  1903, 41229,  4253,  1385,   359,  1545,  9077,
          2165,  1386,   281,   260,  2443,    28,   715,   347, 26538,  9077,
          7349,   284, 34080,   282, 16375,    30,  1216,  2165,   803,   288,
           260, 15879,  5264,   282,   260,  6621,    30,   198,   198,  3757,
          7777,   284, 18688,    42,   198,   198,   504,  2443,   314,   253,
         12999, 27079,   282,   253, 15879,  5182,  6621,    30,   378,  5861,
         27079,   282,   260, 21301,    28,   260,  6333,    28,   284,   260,
         28557,  6734,  5593,   253,  2588,   282, 21539,   284, 10434,    30,
           378,   722,   282,  2380,   284,  8703, 12666,   260,  9184,   977,
           282,   260,  6621,    30,   198,   198,   504,  2443,   314,  2003,
           599,   282,   253,  3227,  8149,    28,   347,   260,  4054,   284,
          3870,   359,  5707,   351,   260,  5535,   284,  5049,   282,   260,
         15879,  3562,    30,   378,  5861, 27079,   282,   260, 21301,   284,
           260,  6333,    23,  3813,  4567,   338,   260,  5182,   314,   253,
          1546,   284, 13865,  3847,   281,   260,  1977,    30,   198,   198,
          3757, 11466,    42,   198,   198,   504,  2443,   314,   253,  3428,
           284,  5861, 27079,   282,   253, 15879,  5182,  6621,    30,   378,
          5861, 27079,   282,   260, 21301,    28,   260,  6333,    28,   284,
           260, 28557,  6734,  7845,   288,   260, 18061,   284,  9184,   977,
           282,   260,  6621,    30,   378,   722,   282,  2380,   284,  8703,
         12666,   260,  9184,   977]][0][:475]

tknR = [
    1, 11126, 42, 1978, 346, 5125, 260, 2443, 47, 49154, 198, 9519, 9531, 42, 378, 2443, 21559, 253, 6621, 429, 253, 15879, 3562, 28, 4170, 253, 15879, 5182, 6621, 30, 378, 4054, 314, 253, 21301, 28, 837, 827, 15713, 16328, 359, 6934, 281, 7604, 30, 378, 21301, 314, 4412, 351, 1461, 2728, 28, 1285, 42, 1116, 33, 30, 1903, 38257, 370, 4253, 378, 2443, 2744, 827, 1507, 16328, 28, 582, 335, 260, 2049, 284, 260, 550, 335, 260, 1048, 30, 3768, 6237, 314, 8168, 282, 3824, 6333, 28, 511, 9031, 351, 896, 1995, 282, 7921, 284, 23812, 30, 378, 6333, 359, 6934, 281, 7604, 28, 351, 634, 282, 601, 17442, 281, 20470, 7604, 28, 979, 1449, 359, 6934, 281, 540, 9964, 34837, 30, 1116, 34, 30, 1903, 1882, 31146, 284, 11889, 270, 4253, 378, 6333, 359, 9031, 351, 253, 3175, 282, 7921, 28, 1285, 29871, 28, 45540, 28, 16418, 28, 284, 30553, 30, 378, 23812, 502, 359, 9064, 314, 597, 9771, 28, 351, 634, 6333, 9064, 29003, 28, 1248, 23812, 28, 284, 30553, 30, 1116, 35, 30, 1903, 54, 403, 7265, 6577, 756, 4253, 533, 260, 3969, 28, 665, 359, 1545, 28557, 6734, 28, 1285, 26225, 28, 31721, 20273, 28, 284, 550, 26271, 7095, 30, 1216, 6734, 359, 441, 3319, 2773, 281, 260, 5182, 564, 803, 288, 260, 28557, 5264, 282, 260, 6621, 30, 1116, 36, 30, 1903, 42284, 4385, 4627, 4253, 378, 21301, 314, 253, 4749, 28, 1440, 1557, 351, 253, 3175, 282, 12979, 3004, 28, 1285, 12610, 28, 20232, 28, 284, 6495, 30, 378, 2118, 314, 19425, 28, 351, 634, 1721, 282, 2375, 5372, 284, 1449, 282, 1759, 28, 19284, 2763, 30, 1116, 37, 30, 1903, 45123, 4253, 378, 6376, 314, 13178, 347, 253, 2437, 1194, 28, 351, 787, 10695, 355, 5249, 30, 378, 2388, 314, 23952, 26388, 28, 17462, 253, 3091, 14654, 690, 260, 21301, 30, 1116, 38, 30, 1903, 41229, 4253, 1385, 359, 1545, 9077, 2165, 6178, 281, 260, 2443, 28, 715, 347, 26538, 9077, 7349, 284, 34080, 282, 16375, 30, 1216, 2165, 803, 288, 260, 15879, 5264, 282, 260, 6621, 30, 198, 198, 3757, 7777, 284, 18688, 42, 198, 198, 504, 2443, 314, 253, 12999, 27079, 282, 253, 15879, 5182, 6621, 30, 378, 5861, 27079, 282, 260, 21301, 28, 260, 6333, 28, 284, 260, 28557, 6734, 7845, 288, 260, 18061, 2174, 282, 260, 6621, 30, 378, 722, 282, 2380, 284, 8703, 5593, 253, 2588, 282, 5856, 284, 5264, 28, 1625, 260, 6621, 1407, 540, 11198, 284, 18061, 30, 198, 198, 3757, 11466, 42, 198, 198, 504, 2443, 20660, 260, 9615, 282, 253, 15879, 5182, 28, 351, 624, 14035, 3964, 28, 5861, 27079, 282, 260, 21301, 28, 284, 28557, 2728, 30, 378, 5861, 27079, 282, 260, 6333, 284, 480, 3813, 28, 347, 876, 347, 260, 28557, 6734, 284, 9077, 2165, 28, 511, 3578, 288, 260, 18061, 2174, 282, 260, 6621, 30
][:475]


print(len(tknP), len(tknR))
print("Token same?", tknP == tknR)

for i,(p,r) in enumerate(zip(tknP, tknR)):
    if p != r:
        print(f"Token mismatch: Python {p} != Rust {r} at {i}")
        break
    
print(tknP[194], tknR[194])

for i in range(0,400):
    print(processor.tokenizer.decode([tknP[i]])+("#" if i != 195 else "<"), end="")
print()
for i in range(0,400):
    print(processor.tokenizer.decode([tknR[i]])+("#" if i != 195 else "<"), end="")

Token same? True
Python version invariant of seed? True
475 475
Token same? False
Token mismatch: Python 715 != Rust 1285 at 194
715 1285
<|im_start|>#User#:# Can# you# describe# the# image#?#<end_of_utterance>#
#Ass#istant#:# The# image# depicts# a# scene# from# a# fantasy# novel#,# specifically# a# fantasy# battle# scene#.# The# setting# is# a# battlefield#,# where# two# opposing# armies# are# engaged# in# combat#.# The# battlefield# is# filled# with# various# elements#,# including#:#

#1#.# **#Arm#ies#**:# The# image# shows# two# large# armies#,# one# on# the# left# and# the# other# on# the# right#.# Each# army# is# composed# of# numerous# soldiers#,# all# equipped# with# different# types# of# weapons# and# armor#.# The# soldiers# are# engaged# in# combat#,# with# some# of# them# locked# in# fierce# combat#,# while# others# are# engaged# in# more# subtle# maneuvers#.#

#2#.# **#We#apons# and# Arm#or#**:# The# soldiers# are# equipped# with# a# variety# of# weapons#,# including# sword

In [37]:
print(processor.tokenizer.chat_template)
"""<|im_start|>{% for message in messages %}{{message['role'] | capitalize}}{% if message['content'][0]['type'] == 'image' %}{{':'}}{% else %}{{': '}}{% endif %}{% for line in message['content'] %}{% if line['type'] == 'text' %}{{line['text']}}{% elif line['type'] == 'image' %}{{ '<image>' }}{% endif %}{% endfor %}<end_of_utterance>{% endfor %}{% if add_generation_prompt %}{{ 'Assistant:' }}{% endif %}"""

"""
<|im_start|>
{% for message in messages %}
    {{message['role'] | capitalize}}
    {% if message['content'][0]['type'] == 'image' %}
        {{':'}}
    {% else %}
        {{': '}}
    {% endif %}
    {% for line in message['content'] %}
        {% if line['type'] == 'text' %}
            {{line['text']}}
        {% elif line['type'] == 'image' %}
            {{ '<image>' }}
        {% endif %}
    {% endfor %}
    <end_of_utterance>]
{% endfor %}
{% if add_generation_prompt %}
    {{ 'Assistant:' }}
{% endif %}
"""

<|im_start|>{% for message in messages %}{{message['role'] | capitalize}}{% if message['content'][0]['type'] == 'image' %}{{':'}}{% else %}{{': '}}{% endif %}{% for line in message['content'] %}{% if line['type'] == 'text' %}{{line['text']}}{% elif line['type'] == 'image' %}{{ '<image>' }}{% endif %}{% endfor %}<end_of_utterance>
{% endfor %}{% if add_generation_prompt %}{{ 'Assistant:' }}{% endif %}


In [ ]:
"""
[  0] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00498791, MAE:0.05653022) 
[  1] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00345520, MAE:0.04663197) 
[  2] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00743339, MAE:0.07156506) 
[  3] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00258058, MAE:0.04027143) 
[  4] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00471779, MAE:0.05765667) 
[  5] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00214532, MAE:0.03561436) 
[  6] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00717968, MAE:0.07259891) 
[  7] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00240679, MAE:0.03841574) 
[  8] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00267795, MAE:0.03917438) 
[  9] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00341212, MAE:0.04623746) 
[ 10] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00655282, MAE:0.06964781) 
[ 11] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00337755, MAE:0.04528118) 
[ 12] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00346823, MAE:0.04677764) 
[ 13] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00215553, MAE:0.03629105) 
[ 14] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00210159, MAE:0.03621835) 
[ 15] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00209621, MAE:0.03601030) 
[ 16] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00458014, MAE:0.05751910) 
[ 17] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00321537, MAE:0.04457099) 
[ 18] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00221294, MAE:0.03706241) 
[ 19] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00230326, MAE:0.03753873) 
[ 20] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00490775, MAE:0.05517812) 
[ 21] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00253869, MAE:0.03951226) 
[ 22] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00185191, MAE:0.03371896) 
[ 23] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00245185, MAE:0.03869445) 
[ 24] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00158067, MAE:0.03089164) 
[ 25] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00230192, MAE:0.03896976) 
[ 26] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00203481, MAE:0.03505059) 
[ 27] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00388136, MAE:0.05035483) 
[ 28] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00298547, MAE:0.04244272) 
[ 29] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00418711, MAE:0.05119276) 
[ 30] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00678076, MAE:0.06796249) 
[ 31] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00422022, MAE:0.05239927) 
[ 32] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00406671, MAE:0.05050832) 
[ 33] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00603566, MAE:0.06305166) 
[ 34] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00352726, MAE:0.04656101) 
[ 35] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00257460, MAE:0.03900414) 
[ 36] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00224177, MAE:0.03649930) 
[ 37] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00186132, MAE:0.03284306) 
[ 38] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00362886, MAE:0.04659612) 
[ 39] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00318316, MAE:0.04389110) 
[ 40] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00385028, MAE:0.04851726) 
[ 41] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00357628, MAE:0.04711151) 
[ 42] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00269444, MAE:0.04064129) 
[ 43] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00457270, MAE:0.05337163) 
[ 44] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00622645, MAE:0.06259437) 
[ 45] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00338396, MAE:0.04473471) 
[ 46] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00404746, MAE:0.04943856) 
[ 47] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00598386, MAE:0.06133371) 
[ 48] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00263827, MAE:0.03977899) 
[ 49] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.00282223, MAE:0.04069102) 
[ 50] EMBEDS(MSE:0.00328064, MAE:0.04541016)  LOGITS(MSE:4.75695705, MAE:1.72113216) 
[ 51] EMBEDS(MSE:0.00781250, MAE:0.07080078)  LOGITS(MSE:9.31803608, MAE:2.41065550) 
[ 52] EMBEDS(MSE:0.00561523, MAE:0.05517578)  LOGITS(MSE:8.49586868, MAE:2.30196118) 
[ 53] EMBEDS(MSE:0.00842285, MAE:0.07373047)  LOGITS(MSE:9.05771923, MAE:2.38667345) 
[ 54] EMBEDS(MSE:0.00729370, MAE:0.06689453)  LOGITS(MSE:8.70569611, MAE:2.36143899) 
[ 55] EMBEDS(MSE:0.00817871, MAE:0.07177734)  LOGITS(MSE:8.70968914, MAE:2.33805823) 
[ 56] EMBEDS(MSE:0.00744629, MAE:0.06542969)  LOGITS(MSE:8.61667919, MAE:2.33717394) 
[ 57] EMBEDS(MSE:0.00656128, MAE:0.06201172)  LOGITS(MSE:7.94118500, MAE:2.23595548) 
[ 58] EMBEDS(MSE:0.00759888, MAE:0.06640625)  LOGITS(MSE:1.89121437, MAE:1.08155644) 
[ 59] EMBEDS(MSE:0.00811768, MAE:0.07177734)  LOGITS(MSE:13.58058167, MAE:2.95576048) 
[ 60] EMBEDS(MSE:0.00830078, MAE:0.07177734)  LOGITS(MSE:8.93497467, MAE:2.40020084) 
[ 61] EMBEDS(MSE:0.00646973, MAE:0.06396484)  LOGITS(MSE:6.42452335, MAE:2.00281501) 
[ 62] EMBEDS(MSE:0.00555420, MAE:0.05908203)  LOGITS(MSE:8.13954163, MAE:2.24142218) 
[ 63] EMBEDS(MSE:0.00817871, MAE:0.07177734)  LOGITS(MSE:9.70573902, MAE:2.48770595) 
[ 64] EMBEDS(MSE:0.00439453, MAE:0.05126953)  LOGITS(MSE:5.27534342, MAE:1.79118228) 
[ 65] EMBEDS(MSE:0.00674438, MAE:0.06396484)  LOGITS(MSE:6.35532904, MAE:2.01777959) 
[ 66] EMBEDS(MSE:0.00891113, MAE:0.07373047)  LOGITS(MSE:6.99130535, MAE:2.09858584) 
[ 67] EMBEDS(MSE:0.00823975, MAE:0.06787109)  LOGITS(MSE:7.86230946, MAE:2.19107604) 
[ 68] EMBEDS(MSE:0.00573730, MAE:0.04296875)  LOGITS(MSE:6.09115362, MAE:1.95695865) 
[ 69] EMBEDS(MSE:0.00524902, MAE:0.05078125)  LOGITS(MSE:7.40646410, MAE:2.14035988) 
[ 70] EMBEDS(MSE:0.00537109, MAE:0.05371094)  LOGITS(MSE:17.08960533, MAE:3.39373231) 
[ 71] EMBEDS(MSE:0.00897217, MAE:0.06982422)  LOGITS(MSE:19.53756332, MAE:3.57274628) 
[ 72] EMBEDS(MSE:0.00970459, MAE:0.07861328)  LOGITS(MSE:12.04500294, MAE:2.76010132) 
[ 73] EMBEDS(MSE:0.00964355, MAE:0.07861328)  LOGITS(MSE:5.39363813, MAE:1.83514500) 
[ 74] EMBEDS(MSE:0.00738525, MAE:0.06835938)  LOGITS(MSE:7.09322596, MAE:2.09815025) 
[ 75] EMBEDS(MSE:0.00738525, MAE:0.06689453)  LOGITS(MSE:12.66374016, MAE:2.95276070) 
[ 76] EMBEDS(MSE:0.00543213, MAE:0.05664062)  LOGITS(MSE:14.48589897, MAE:3.15480781) 
[ 77] EMBEDS(MSE:0.00631714, MAE:0.06298828)  LOGITS(MSE:7.00163507, MAE:2.09377956) 
[ 78] EMBEDS(MSE:0.00592041, MAE:0.06127930)  LOGITS(MSE:4.98612499, MAE:1.76484299) 
[ 79] EMBEDS(MSE:0.00805664, MAE:0.07031250)  LOGITS(MSE:7.78036928, MAE:2.19643855) 
[ 80] EMBEDS(MSE:0.00811768, MAE:0.07128906)  LOGITS(MSE:12.62517262, MAE:2.85733581) 
[ 81] EMBEDS(MSE:0.00741577, MAE:0.06591797)  LOGITS(MSE:5.27401066, MAE:1.82250190) 
[ 82] EMBEDS(MSE:0.00637817, MAE:0.05981445)  LOGITS(MSE:10.60338974, MAE:2.58467603) 
[ 83] EMBEDS(MSE:0.00811768, MAE:0.07128906)  LOGITS(MSE:13.91224194, MAE:3.00695753) 
[ 84] EMBEDS(MSE:0.00683594, MAE:0.06542969)  LOGITS(MSE:7.00413609, MAE:2.08859086) 
[ 85] EMBEDS(MSE:0.00582886, MAE:0.05932617)  LOGITS(MSE:9.29948330, MAE:2.42812586) 
[ 86] EMBEDS(MSE:0.00830078, MAE:0.06933594)  LOGITS(MSE:9.20536041, MAE:2.39627051) 
[ 87] EMBEDS(MSE:0.00439453, MAE:0.04052734)  LOGITS(MSE:7.33167028, MAE:2.13985300) 
[ 88] EMBEDS(MSE:0.00988770, MAE:0.07910156)  LOGITS(MSE:2.59173632, MAE:1.27353585) 
[ 89] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:1.24049020, MAE:0.88008291) 
[ 90] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:0.90615797, MAE:0.75383765) 
[ 91] EMBEDS(MSE:0.00671387, MAE:0.06396484)  LOGITS(MSE:5.64091778, MAE:1.89211547) 
[ 92] EMBEDS(MSE:0.00674438, MAE:0.06396484)  LOGITS(MSE:7.22509432, MAE:2.14812398) 
[ 93] EMBEDS(MSE:0.00891113, MAE:0.07373047)  LOGITS(MSE:5.99499273, MAE:1.94210327) 
[ 94] EMBEDS(MSE:0.00823975, MAE:0.06787109)  LOGITS(MSE:9.24210262, MAE:2.46235895) 
[ 95] EMBEDS(MSE:0.00573730, MAE:0.04296875)  LOGITS(MSE:4.72067356, MAE:1.70888543) 
[ 96] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:5.50275278, MAE:1.82837343) 
[ 97] EMBEDS(MSE:0.00579834, MAE:0.05981445)  LOGITS(MSE:8.96831036, MAE:2.35260773) 
[ 98] EMBEDS(MSE:0.00631714, MAE:0.05859375)  LOGITS(MSE:33.49641037, MAE:4.78348160) 
[ 99] EMBEDS(MSE:0.00823975, MAE:0.07177734)  LOGITS(MSE:14.04784203, MAE:2.93083334) 
[100] EMBEDS(MSE:0.00830078, MAE:0.07275391)  LOGITS(MSE:10.21409702, MAE:2.52531958) 
[101] EMBEDS(MSE:0.00708008, MAE:0.06640625)  LOGITS(MSE:10.10577869, MAE:2.49931288) 
[102] EMBEDS(MSE:0.00494385, MAE:0.05541992)  LOGITS(MSE:11.40822506, MAE:2.66389728) 
[103] EMBEDS(MSE:0.00909424, MAE:0.07519531)  LOGITS(MSE:14.46766186, MAE:2.98153281) 
[104] EMBEDS(MSE:0.00976562, MAE:0.07812500)  LOGITS(MSE:5.54061174, MAE:1.86484158) 
[105] EMBEDS(MSE:0.00448608, MAE:0.03930664)  LOGITS(MSE:13.01215839, MAE:2.84626508) 
[106] EMBEDS(MSE:0.00842285, MAE:0.06738281)  LOGITS(MSE:17.37368965, MAE:3.27588820) 
[107] EMBEDS(MSE:0.00842285, MAE:0.06835938)  LOGITS(MSE:23.20389557, MAE:3.71930623) 
[108] EMBEDS(MSE:0.00891113, MAE:0.07568359)  LOGITS(MSE:16.43590355, MAE:3.24771953) 
[109] EMBEDS(MSE:0.00805664, MAE:0.07080078)  LOGITS(MSE:14.05100441, MAE:2.95304036) 
[110] EMBEDS(MSE:0.00854492, MAE:0.07324219)  LOGITS(MSE:25.37744904, MAE:4.23655891) 
[111] EMBEDS(MSE:0.00854492, MAE:0.07324219)  LOGITS(MSE:26.83743286, MAE:4.43753958) 
[112] EMBEDS(MSE:0.00805664, MAE:0.07080078)  LOGITS(MSE:13.54452515, MAE:2.98565316) 
[113] EMBEDS(MSE:0.00781250, MAE:0.06982422)  LOGITS(MSE:18.79349518, MAE:3.64775085) 
[114] EMBEDS(MSE:0.00698853, MAE:0.06176758)  LOGITS(MSE:13.09057331, MAE:2.82793832) 
[115] EMBEDS(MSE:0.00628662, MAE:0.05932617)  LOGITS(MSE:10.58152103, MAE:2.66691208) 
[116] EMBEDS(MSE:0.00781250, MAE:0.07080078)  LOGITS(MSE:16.42959976, MAE:3.30752778) 
[117] EMBEDS(MSE:0.00842285, MAE:0.06982422)  LOGITS(MSE:7.20896912, MAE:2.11141825) 
[118] EMBEDS(MSE:0.00640869, MAE:0.06298828)  LOGITS(MSE:7.01729345, MAE:2.08016014) 
[119] EMBEDS(MSE:0.00573730, MAE:0.05761719)  LOGITS(MSE:8.90091133, MAE:2.35651350) 
[120] EMBEDS(MSE:0.00543213, MAE:0.05688477)  LOGITS(MSE:19.21446991, MAE:3.51852632) 
[121] EMBEDS(MSE:0.00915527, MAE:0.07568359)  LOGITS(MSE:11.55310726, MAE:2.69227576) 
[122] EMBEDS(MSE:0.00741577, MAE:0.06738281)  LOGITS(MSE:9.62590408, MAE:2.45224524) 
[123] EMBEDS(MSE:0.00653076, MAE:0.06396484)  LOGITS(MSE:10.77960873, MAE:2.59062719) 
[124] EMBEDS(MSE:0.00610352, MAE:0.06176758)  LOGITS(MSE:12.62848854, MAE:2.80575943) 
[125] EMBEDS(MSE:0.00552368, MAE:0.05273438)  LOGITS(MSE:21.37378120, MAE:3.67297292) 
[126] EMBEDS(MSE:0.00885010, MAE:0.07519531)  LOGITS(MSE:10.95466518, MAE:2.62097383) 
[127] EMBEDS(MSE:0.00799561, MAE:0.07128906)  LOGITS(MSE:8.75632000, MAE:2.33647275) 
[128] EMBEDS(MSE:0.00573730, MAE:0.04296875)  LOGITS(MSE:16.21152687, MAE:3.31454778) 
[129] EMBEDS(MSE:0.00662231, MAE:0.06494141)  LOGITS(MSE:8.76718044, MAE:2.33784914) 
[130] EMBEDS(MSE:0.00308228, MAE:0.03417969)  LOGITS(MSE:39.67441940, MAE:5.43831873) 
[131] EMBEDS(MSE:0.00830078, MAE:0.07177734)  LOGITS(MSE:13.37122917, MAE:2.91355276) 
[132] EMBEDS(MSE:0.00775146, MAE:0.06591797)  LOGITS(MSE:12.34325600, MAE:2.79483008) 
[133] EMBEDS(MSE:0.00775146, MAE:0.06982422)  LOGITS(MSE:9.49369907, MAE:2.43739939) 
[134] EMBEDS(MSE:0.00306702, MAE:0.04296875)  LOGITS(MSE:10.40474606, MAE:2.56981850) 
[135] EMBEDS(MSE:0.00927734, MAE:0.07617188)  LOGITS(MSE:15.46393776, MAE:3.13304090) 
[136] EMBEDS(MSE:0.00958252, MAE:0.07812500)  LOGITS(MSE:6.34118748, MAE:1.99892080) 
[137] EMBEDS(MSE:0.00369263, MAE:0.04541016)  LOGITS(MSE:22.98148918, MAE:4.05722475) 
[138] EMBEDS(MSE:0.00607300, MAE:0.06176758)  LOGITS(MSE:9.01642513, MAE:2.43616819) 
[139] EMBEDS(MSE:0.00781250, MAE:0.07080078)  LOGITS(MSE:14.36070824, MAE:3.08094621) 
[140] EMBEDS(MSE:0.00830078, MAE:0.07226562)  LOGITS(MSE:13.65335655, MAE:2.93600845) 
[141] EMBEDS(MSE:0.00921631, MAE:0.07568359)  LOGITS(MSE:6.33441782, MAE:1.97495019) 
[142] EMBEDS(MSE:0.00698853, MAE:0.06591797)  LOGITS(MSE:4.86638832, MAE:1.74178636) 
[143] EMBEDS(MSE:0.00506592, MAE:0.05224609)  LOGITS(MSE:14.86790466, MAE:3.19896698) 
[144] EMBEDS(MSE:0.00781250, MAE:0.07080078)  LOGITS(MSE:11.33802605, MAE:2.65854955) 
[145] EMBEDS(MSE:0.00793457, MAE:0.07031250)  LOGITS(MSE:17.28697586, MAE:3.28350186) 
[146] EMBEDS(MSE:0.00823975, MAE:0.07128906)  LOGITS(MSE:8.04561710, MAE:2.24855804) 
[147] EMBEDS(MSE:0.00430298, MAE:0.04980469)  LOGITS(MSE:6.61284351, MAE:2.03784275) 
[148] EMBEDS(MSE:0.00369263, MAE:0.04272461)  LOGITS(MSE:14.74422359, MAE:3.07970548) 
[149] EMBEDS(MSE:0.00787354, MAE:0.07031250)  LOGITS(MSE:8.39802551, MAE:2.28959012) 
[150] EMBEDS(MSE:0.00677490, MAE:0.06542969)  LOGITS(MSE:11.34914303, MAE:2.63855124) 
[151] EMBEDS(MSE:0.00848389, MAE:0.07177734)  LOGITS(MSE:4.89046526, MAE:1.74244368) 
[152] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:3.64838195, MAE:1.51729608) 
[153] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:6.76329231, MAE:2.01666355) 
[154] EMBEDS(MSE:0.00640869, MAE:0.06298828)  LOGITS(MSE:6.20381451, MAE:2.00786066) 
[155] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:4.25323439, MAE:1.61230290) 
[156] EMBEDS(MSE:0.00683594, MAE:0.06445312)  LOGITS(MSE:3.00738168, MAE:1.36906946) 
[157] EMBEDS(MSE:0.00288391, MAE:0.04248047)  LOGITS(MSE:8.62100124, MAE:2.31307983) 
[158] EMBEDS(MSE:0.00561523, MAE:0.05151367)  LOGITS(MSE:11.90698051, MAE:2.74995780) 
[159] EMBEDS(MSE:0.00836182, MAE:0.07226562)  LOGITS(MSE:13.68853664, MAE:2.92771339) 
[160] EMBEDS(MSE:0.00836182, MAE:0.06787109)  LOGITS(MSE:42.40856552, MAE:5.53655910) 
[161] EMBEDS(MSE:0.00817871, MAE:0.07226562)  LOGITS(MSE:13.87268353, MAE:2.94045115) 
[162] EMBEDS(MSE:0.00811768, MAE:0.07128906)  LOGITS(MSE:9.55531597, MAE:2.44056106) 
[163] EMBEDS(MSE:0.00793457, MAE:0.06494141)  LOGITS(MSE:15.88612556, MAE:3.19429231) 
[164] EMBEDS(MSE:0.00515747, MAE:0.04833984)  LOGITS(MSE:15.20572472, MAE:3.07105875) 
[165] EMBEDS(MSE:0.00775146, MAE:0.06445312)  LOGITS(MSE:14.65648556, MAE:3.01682234) 
[166] EMBEDS(MSE:0.00622559, MAE:0.06225586)  LOGITS(MSE:10.54470444, MAE:2.56181288) 
[167] EMBEDS(MSE:0.00439453, MAE:0.04052734)  LOGITS(MSE:9.81571388, MAE:2.46499300) 
[168] EMBEDS(MSE:0.00817871, MAE:0.07226562)  LOGITS(MSE:8.33978844, MAE:2.28011942) 
[169] EMBEDS(MSE:0.00662231, MAE:0.06250000)  LOGITS(MSE:10.29512501, MAE:2.52095985) 
[170] EMBEDS(MSE:0.00686646, MAE:0.06640625)  LOGITS(MSE:7.50652218, MAE:2.15265942) 
[171] EMBEDS(MSE:0.00711060, MAE:0.06640625)  LOGITS(MSE:11.84175301, MAE:2.76571369) 
[172] EMBEDS(MSE:0.00543213, MAE:0.05859375)  LOGITS(MSE:9.08322716, MAE:2.44807267) 
[173] EMBEDS(MSE:0.00503540, MAE:0.05517578)  LOGITS(MSE:28.51735306, MAE:4.68492842) 
[174] EMBEDS(MSE:0.00952148, MAE:0.07714844)  LOGITS(MSE:13.39418888, MAE:3.03529119) 
[175] EMBEDS(MSE:0.00756836, MAE:0.06689453)  LOGITS(MSE:3.06445360, MAE:1.38102555) 
[176] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:2.36153197, MAE:1.21016383) 
[177] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:8.17104340, MAE:2.29735208) 
[178] EMBEDS(MSE:0.00787354, MAE:0.07031250)  LOGITS(MSE:14.20188141, MAE:3.09773159) 
[179] EMBEDS(MSE:0.00830078, MAE:0.06933594)  LOGITS(MSE:8.85597610, MAE:2.37627029) 
[180] EMBEDS(MSE:0.00686646, MAE:0.06494141)  LOGITS(MSE:10.50463867, MAE:2.58740854) 
[181] EMBEDS(MSE:0.00308228, MAE:0.03417969)  LOGITS(MSE:8.10951233, MAE:2.23616552) 
[182] EMBEDS(MSE:0.00769043, MAE:0.06738281)  LOGITS(MSE:12.69240379, MAE:2.81446528) 
[183] EMBEDS(MSE:0.00695801, MAE:0.06591797)  LOGITS(MSE:7.50562525, MAE:2.14995241) 
[184] EMBEDS(MSE:0.00698853, MAE:0.06591797)  LOGITS(MSE:4.55932856, MAE:1.70029652) 
[185] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:3.34445238, MAE:1.44031036) 
[186] EMBEDS(MSE:0.00616455, MAE:0.06176758)  LOGITS(MSE:11.87526703, MAE:2.72663164) 
[187] EMBEDS(MSE:0.00878906, MAE:0.07324219)  LOGITS(MSE:8.98953724, MAE:2.38012981) 
[188] EMBEDS(MSE:0.00759888, MAE:0.06738281)  LOGITS(MSE:5.79156685, MAE:1.87673175) 
[189] EMBEDS(MSE:0.00509644, MAE:0.05615234)  LOGITS(MSE:9.99382591, MAE:2.50357103) 
[190] EMBEDS(MSE:0.00762939, MAE:0.06835938)  LOGITS(MSE:7.71816206, MAE:2.19708323) 
[191] EMBEDS(MSE:0.00735474, MAE:0.06787109)  LOGITS(MSE:12.11656475, MAE:2.72612619) 
[192] EMBEDS(MSE:0.00970459, MAE:0.07714844)  LOGITS(MSE:9.78929615, MAE:2.48165703) 
[193] EMBEDS(MSE:0.00811768, MAE:0.06591797)  LOGITS(MSE:7.88085890, MAE:2.18599391) 
[194] EMBEDS(MSE:0.00570679, MAE:0.05346680)  LOGITS(MSE:10.15519428, MAE:2.48321819) 
[195] EMBEDS(MSE:0.00668335, MAE:0.05883789)  LOGITS(MSE:15.54814148, MAE:3.06887102) 
[196] EMBEDS(MSE:0.00717163, MAE:0.06640625)  LOGITS(MSE:11.44870567, MAE:2.70905209) 
[197] EMBEDS(MSE:0.00637817, MAE:0.06225586)  LOGITS(MSE:16.33310509, MAE:3.17088175) 
[198] EMBEDS(MSE:0.00878906, MAE:0.07421875)  LOGITS(MSE:12.60156536, MAE:2.84715223) 
[199] EMBEDS(MSE:0.00683594, MAE:0.06396484)  LOGITS(MSE:11.47397041, MAE:2.69914198) 
[200] EMBEDS(MSE:0.00732422, MAE:0.06738281)  LOGITS(MSE:10.75962257, MAE:2.60071468) 
[201] EMBEDS(MSE:0.00756836, MAE:0.06933594)  LOGITS(MSE:9.80392838, MAE:2.46214080) 
[202] EMBEDS(MSE:0.00787354, MAE:0.07080078)  LOGITS(MSE:8.97429848, MAE:2.38152075) 
[203] EMBEDS(MSE:0.00656128, MAE:0.06494141)  LOGITS(MSE:7.86621284, MAE:2.20576930) 
[204] EMBEDS(MSE:0.00656128, MAE:0.06347656)  LOGITS(MSE:6.70895243, MAE:2.04767537) 
[205] EMBEDS(MSE:0.00573730, MAE:0.04296875)  LOGITS(MSE:10.42418861, MAE:2.57265162) 
[206] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:6.02130508, MAE:1.94114089) 
[207] EMBEDS(MSE:0.00659180, MAE:0.05834961)  LOGITS(MSE:21.76041603, MAE:3.68089080) 
[208] EMBEDS(MSE:0.00830078, MAE:0.07275391)  LOGITS(MSE:21.29532242, MAE:3.88558221) 
[209] EMBEDS(MSE:0.00860596, MAE:0.07421875)  LOGITS(MSE:11.36249352, MAE:2.68994379) 
[210] EMBEDS(MSE:0.00698853, MAE:0.06298828)  LOGITS(MSE:11.49793720, MAE:2.72568560) 
[211] EMBEDS(MSE:0.00711060, MAE:0.06640625)  LOGITS(MSE:24.34324074, MAE:4.21220779) 
[212] EMBEDS(MSE:0.00619507, MAE:0.05444336)  LOGITS(MSE:20.02232552, MAE:3.57247353) 
[213] EMBEDS(MSE:0.00738525, MAE:0.06787109)  LOGITS(MSE:5.04647732, MAE:1.77758491) 
[214] EMBEDS(MSE:0.00805664, MAE:0.07128906)  LOGITS(MSE:5.07080793, MAE:1.76131165) 
[215] EMBEDS(MSE:0.00366211, MAE:0.03613281)  LOGITS(MSE:4.37387085, MAE:1.63692641) 
[216] EMBEDS(MSE:0.00646973, MAE:0.06396484)  LOGITS(MSE:5.88759422, MAE:1.90212238) 
[217] EMBEDS(MSE:0.00704956, MAE:0.06591797)  LOGITS(MSE:15.31268215, MAE:3.09827852) 
[218] EMBEDS(MSE:0.00903320, MAE:0.07519531)  LOGITS(MSE:8.20132637, MAE:2.25513387) 
[219] EMBEDS(MSE:0.00741577, MAE:0.06738281)  LOGITS(MSE:4.93430901, MAE:1.74895990) 
[220] EMBEDS(MSE:0.00460815, MAE:0.05126953)  LOGITS(MSE:5.44363070, MAE:1.87126756) 
[221] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:2.05402160, MAE:1.13868248) 
[222] EMBEDS(MSE:0.00534058, MAE:0.05810547)  LOGITS(MSE:3.49448490, MAE:1.47017884) 
[223] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:2.71826100, MAE:1.30072153) 
[224] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:2.17516565, MAE:1.16729116) 
[225] EMBEDS(MSE:0.00506592, MAE:0.05224609)  LOGITS(MSE:4.72128916, MAE:1.77142823) 
[226] EMBEDS(MSE:0.00646973, MAE:0.06396484)  LOGITS(MSE:4.70527792, MAE:1.71369946) 
[227] EMBEDS(MSE:0.00534058, MAE:0.05810547)  LOGITS(MSE:2.61768198, MAE:1.27519011) 
[228] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:3.38632512, MAE:1.42439222) 
[229] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:3.39252234, MAE:1.46362150) 
[230] EMBEDS(MSE:0.00506592, MAE:0.05224609)  LOGITS(MSE:1.77237380, MAE:1.04484916) 
[231] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:3.39928341, MAE:1.45654404) 
[232] EMBEDS(MSE:0.00534058, MAE:0.05810547)  LOGITS(MSE:2.67931771, MAE:1.28956687) 
[233] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:4.62567186, MAE:1.67468262) 
[234] EMBEDS(MSE:0.00288391, MAE:0.04248047)  LOGITS(MSE:20.70992851, MAE:3.88087296) 
[235] EMBEDS(MSE:0.00588989, MAE:0.06054688)  LOGITS(MSE:6.37643528, MAE:1.97585762) 
[236] EMBEDS(MSE:0.00656128, MAE:0.06201172)  LOGITS(MSE:6.79462290, MAE:2.05825520) 
[237] EMBEDS(MSE:0.00714111, MAE:0.06640625)  LOGITS(MSE:9.85047722, MAE:2.49274468) 
[238] EMBEDS(MSE:0.00823975, MAE:0.07177734)  LOGITS(MSE:8.85967255, MAE:2.38891077) 
[239] EMBEDS(MSE:0.00759888, MAE:0.06738281)  LOGITS(MSE:8.75288010, MAE:2.33297706) 
[240] EMBEDS(MSE:0.00561523, MAE:0.05883789)  LOGITS(MSE:6.12522316, MAE:1.94661558) 
[241] EMBEDS(MSE:0.00897217, MAE:0.07470703)  LOGITS(MSE:7.07230568, MAE:2.10852718) 
[242] EMBEDS(MSE:0.00848389, MAE:0.07324219)  LOGITS(MSE:12.20428276, MAE:2.78520107) 
[243] EMBEDS(MSE:0.00787354, MAE:0.07128906)  LOGITS(MSE:12.04011154, MAE:2.75668621) 
[244] EMBEDS(MSE:0.00558472, MAE:0.05908203)  LOGITS(MSE:7.06906605, MAE:2.09528899) 
[245] EMBEDS(MSE:0.00466919, MAE:0.05126953)  LOGITS(MSE:24.96692085, MAE:4.29829264) 
[246] EMBEDS(MSE:0.00588989, MAE:0.06054688)  LOGITS(MSE:9.93421745, MAE:2.65875125) 
[247] EMBEDS(MSE:0.00634766, MAE:0.06152344)  LOGITS(MSE:8.73049545, MAE:2.35698628) 
[248] EMBEDS(MSE:0.00714111, MAE:0.06640625)  LOGITS(MSE:8.33641911, MAE:2.29212904) 
[249] EMBEDS(MSE:0.00823975, MAE:0.07177734)  LOGITS(MSE:7.44173956, MAE:2.16647983) 
[250] EMBEDS(MSE:0.00759888, MAE:0.06738281)  LOGITS(MSE:5.34260702, MAE:1.82966065) 
[251] EMBEDS(MSE:0.00613403, MAE:0.06176758)  LOGITS(MSE:5.11411381, MAE:1.75853741) 
[252] EMBEDS(MSE:0.00753784, MAE:0.06884766)  LOGITS(MSE:6.97552395, MAE:2.07462549) 
[253] EMBEDS(MSE:0.00723267, MAE:0.06738281)  LOGITS(MSE:9.41741371, MAE:2.44411802) 
[254] EMBEDS(MSE:0.00823975, MAE:0.07177734)  LOGITS(MSE:9.53287983, MAE:2.49916911) 
[255] EMBEDS(MSE:0.00759888, MAE:0.06738281)  LOGITS(MSE:5.17979336, MAE:1.75379932) 
[256] EMBEDS(MSE:0.00509644, MAE:0.05615234)  LOGITS(MSE:5.05611277, MAE:1.75937212) 
[257] EMBEDS(MSE:0.00668335, MAE:0.06445312)  LOGITS(MSE:9.11661053, MAE:2.42493701) 
[258] EMBEDS(MSE:0.00735474, MAE:0.06787109)  LOGITS(MSE:9.68965435, MAE:2.48243308) 
[259] EMBEDS(MSE:0.00723267, MAE:0.06738281)  LOGITS(MSE:7.49410963, MAE:2.16906548) 
[260] EMBEDS(MSE:0.00759888, MAE:0.06787109)  LOGITS(MSE:9.41905785, MAE:2.45689750) 
[261] EMBEDS(MSE:0.00759888, MAE:0.06738281)  LOGITS(MSE:4.62088251, MAE:1.62795687) 
[262] EMBEDS(MSE:0.00613403, MAE:0.06176758)  LOGITS(MSE:6.42491722, MAE:1.98492050) 
[263] EMBEDS(MSE:0.00668335, MAE:0.06445312)  LOGITS(MSE:7.34326124, MAE:2.14761257) 
[264] EMBEDS(MSE:0.00891113, MAE:0.07373047)  LOGITS(MSE:15.62534428, MAE:3.12460518) 
[265] EMBEDS(MSE:0.00952148, MAE:0.07666016)  LOGITS(MSE:7.71335077, MAE:2.19737124) 
[266] EMBEDS(MSE:0.00683594, MAE:0.06542969)  LOGITS(MSE:5.51766872, MAE:1.84855211) 
[267] EMBEDS(MSE:0.00634766, MAE:0.06347656)  LOGITS(MSE:5.61237288, MAE:1.85284865) 
[268] EMBEDS(MSE:0.00723267, MAE:0.06396484)  LOGITS(MSE:8.94312859, MAE:2.36714816) 
[269] EMBEDS(MSE:0.00607300, MAE:0.05322266)  LOGITS(MSE:7.80008078, MAE:2.20440149) 
[270] EMBEDS(MSE:0.00747681, MAE:0.06201172)  LOGITS(MSE:11.72499752, MAE:2.65328622) 
[271] EMBEDS(MSE:0.00698853, MAE:0.06542969)  LOGITS(MSE:8.44128609, MAE:2.34152627) 
[272] EMBEDS(MSE:0.00564575, MAE:0.05786133)  LOGITS(MSE:5.65827990, MAE:1.90693295) 
[273] EMBEDS(MSE:0.00646973, MAE:0.06396484)  LOGITS(MSE:5.67677164, MAE:1.92394924) 
[274] EMBEDS(MSE:0.00628662, MAE:0.06298828)  LOGITS(MSE:12.92482471, MAE:2.83798814) 
[275] EMBEDS(MSE:0.00842285, MAE:0.07275391)  LOGITS(MSE:9.78664589, MAE:2.52292895) 
[276] EMBEDS(MSE:0.00512695, MAE:0.05566406)  LOGITS(MSE:9.60619068, MAE:2.49524069) 
[277] EMBEDS(MSE:0.00552368, MAE:0.05810547)  LOGITS(MSE:10.54822540, MAE:2.63766980) 
[278] EMBEDS(MSE:0.00817871, MAE:0.07177734)  LOGITS(MSE:8.55737114, MAE:2.33151245) 
[279] EMBEDS(MSE:0.00823975, MAE:0.07275391)  LOGITS(MSE:12.18542862, MAE:2.76889110) 
[280] EMBEDS(MSE:0.00811768, MAE:0.06591797)  LOGITS(MSE:11.28941631, MAE:2.71571183) 
[281] EMBEDS(MSE:0.00570679, MAE:0.05346680)  LOGITS(MSE:16.18205643, MAE:3.31986094) 
[282] EMBEDS(MSE:0.00668335, MAE:0.05883789)  LOGITS(MSE:15.22324562, MAE:3.05486536) 
[283] EMBEDS(MSE:0.00747681, MAE:0.06835938)  LOGITS(MSE:21.11833572, MAE:3.80587554) 
[284] EMBEDS(MSE:0.00811768, MAE:0.07177734)  LOGITS(MSE:13.33939362, MAE:2.89663363) 
[285] EMBEDS(MSE:0.00982666, MAE:0.07910156)  LOGITS(MSE:16.92556953, MAE:3.36387420) 
[286] EMBEDS(MSE:0.00793457, MAE:0.07128906)  LOGITS(MSE:12.64120865, MAE:2.80864668) 
[287] EMBEDS(MSE:0.00726318, MAE:0.06787109)  LOGITS(MSE:10.84227562, MAE:2.59903073) 
[288] EMBEDS(MSE:0.00714111, MAE:0.06396484)  LOGITS(MSE:5.64161253, MAE:1.86274302) 
[289] EMBEDS(MSE:0.00485229, MAE:0.05053711)  LOGITS(MSE:10.69466782, MAE:2.56505084) 
[290] EMBEDS(MSE:0.00717163, MAE:0.06298828)  LOGITS(MSE:17.60948181, MAE:3.28676963) 
[291] EMBEDS(MSE:0.00634766, MAE:0.05957031)  LOGITS(MSE:7.90189457, MAE:2.23505163) 
[292] EMBEDS(MSE:0.00811768, MAE:0.07128906)  LOGITS(MSE:13.05972195, MAE:2.90614700) 
[293] EMBEDS(MSE:0.00729370, MAE:0.06835938)  LOGITS(MSE:7.11423683, MAE:2.09687519) 
[294] EMBEDS(MSE:0.00358582, MAE:0.04443359)  LOGITS(MSE:9.56497192, MAE:2.43857908) 
[295] EMBEDS(MSE:0.00909424, MAE:0.07519531)  LOGITS(MSE:11.20722103, MAE:2.64814973) 
[296] EMBEDS(MSE:0.00738525, MAE:0.06640625)  LOGITS(MSE:8.94735813, MAE:2.34905958) 
[297] EMBEDS(MSE:0.00793457, MAE:0.07128906)  LOGITS(MSE:7.41916180, MAE:2.14895082) 
[298] EMBEDS(MSE:0.00723267, MAE:0.06396484)  LOGITS(MSE:8.94540882, MAE:2.35632992) 
[299] EMBEDS(MSE:0.00653076, MAE:0.05712891)  LOGITS(MSE:13.36003971, MAE:2.91745496) 
[300] EMBEDS(MSE:0.00598145, MAE:0.05126953)  LOGITS(MSE:28.97099495, MAE:4.22609138) 
[301] EMBEDS(MSE:0.00793457, MAE:0.07080078)  LOGITS(MSE:19.11971855, MAE:3.51957464) 
[302] EMBEDS(MSE:0.01068115, MAE:0.08251953)  LOGITS(MSE:9.50547600, MAE:2.42260695) 
[303] EMBEDS(MSE:0.00854492, MAE:0.07324219)  LOGITS(MSE:8.35412884, MAE:2.28716469) 
[304] EMBEDS(MSE:0.00793457, MAE:0.07031250)  LOGITS(MSE:16.01851463, MAE:3.19776344) 
[305] EMBEDS(MSE:0.00668335, MAE:0.06079102)  LOGITS(MSE:8.81800747, MAE:2.35050607) 
[306] EMBEDS(MSE:0.00799561, MAE:0.07031250)  LOGITS(MSE:12.36059666, MAE:2.79330492) 
[307] EMBEDS(MSE:0.00817871, MAE:0.06738281)  LOGITS(MSE:13.35380650, MAE:2.87034583) 
[308] EMBEDS(MSE:0.00692749, MAE:0.05981445)  LOGITS(MSE:22.34596634, MAE:4.05499363) 
[309] EMBEDS(MSE:0.00775146, MAE:0.07031250)  LOGITS(MSE:6.11507225, MAE:1.93120396) 
[310] EMBEDS(MSE:0.00366211, MAE:0.03613281)  LOGITS(MSE:3.56639957, MAE:1.47956765) 
[311] EMBEDS(MSE:0.00646973, MAE:0.06396484)  LOGITS(MSE:3.26528645, MAE:1.41329432) 
[312] EMBEDS(MSE:0.00704956, MAE:0.06591797)  LOGITS(MSE:15.57834625, MAE:3.11373782) 
[313] EMBEDS(MSE:0.00903320, MAE:0.07519531)  LOGITS(MSE:8.13399410, MAE:2.25401878) 
[314] EMBEDS(MSE:0.00741577, MAE:0.06738281)  LOGITS(MSE:3.91033411, MAE:1.55866325) 
[315] EMBEDS(MSE:0.00460815, MAE:0.05126953)  LOGITS(MSE:4.23694324, MAE:1.65675986) 
[316] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:2.44544148, MAE:1.23960698) 
[317] EMBEDS(MSE:0.00534058, MAE:0.05810547)  LOGITS(MSE:3.32419825, MAE:1.43046010) 
[318] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:2.35468554, MAE:1.20572865) 
[319] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:2.34283590, MAE:1.21398938) 
[320] EMBEDS(MSE:0.00515747, MAE:0.05664062)  LOGITS(MSE:6.77948475, MAE:2.05837917) 
[321] EMBEDS(MSE:0.00958252, MAE:0.07812500)  LOGITS(MSE:10.30185699, MAE:2.58021927) 
[322] EMBEDS(MSE:0.00805664, MAE:0.07080078)  LOGITS(MSE:14.61843395, MAE:3.03131557) 
[323] EMBEDS(MSE:0.00842285, MAE:0.07226562)  LOGITS(MSE:6.08596659, MAE:1.92212331) 
[324] EMBEDS(MSE:0.00665283, MAE:0.06542969)  LOGITS(MSE:3.80767608, MAE:1.56178606) 
[325] EMBEDS(MSE:0.00448608, MAE:0.03930664)  LOGITS(MSE:14.72425652, MAE:3.16722488) 
[326] EMBEDS(MSE:0.00717163, MAE:0.06176758)  LOGITS(MSE:9.79274845, MAE:2.49441791) 
[327] EMBEDS(MSE:0.00738525, MAE:0.06152344)  LOGITS(MSE:12.07678604, MAE:2.73107004) 
[328] EMBEDS(MSE:0.00793457, MAE:0.06982422)  LOGITS(MSE:8.22294140, MAE:2.24370742) 
[329] EMBEDS(MSE:0.00430298, MAE:0.04980469)  LOGITS(MSE:7.88108778, MAE:2.24794245) 
[330] EMBEDS(MSE:0.00704956, MAE:0.06640625)  LOGITS(MSE:6.60212326, MAE:2.01447916) 
[331] EMBEDS(MSE:0.00903320, MAE:0.07568359)  LOGITS(MSE:10.99916363, MAE:2.75501823) 
[332] EMBEDS(MSE:0.00646973, MAE:0.06298828)  LOGITS(MSE:15.26056862, MAE:3.09005642) 
[333] EMBEDS(MSE:0.00704956, MAE:0.06201172)  LOGITS(MSE:14.00070190, MAE:3.00206876) 
[334] EMBEDS(MSE:0.00753784, MAE:0.06884766)  LOGITS(MSE:8.15921021, MAE:2.30334854) 
[335] EMBEDS(MSE:0.00421143, MAE:0.04809570)  LOGITS(MSE:8.75214195, MAE:2.33780360) 
[336] EMBEDS(MSE:0.00927734, MAE:0.07666016)  LOGITS(MSE:8.28812981, MAE:2.27780581) 
[337] EMBEDS(MSE:0.00756836, MAE:0.06640625)  LOGITS(MSE:7.39285803, MAE:2.11814356) 
[338] EMBEDS(MSE:0.00772095, MAE:0.06982422)  LOGITS(MSE:5.42499924, MAE:1.82355976) 
[339] EMBEDS(MSE:0.00598145, MAE:0.05883789)  LOGITS(MSE:4.78612423, MAE:1.70805490) 
[340] EMBEDS(MSE:0.00671387, MAE:0.06079102)  LOGITS(MSE:11.16347122, MAE:2.65052342) 
[341] EMBEDS(MSE:0.00671387, MAE:0.05883789)  LOGITS(MSE:7.00483656, MAE:2.08770466) 
[342] EMBEDS(MSE:0.00793457, MAE:0.06591797)  LOGITS(MSE:8.48985767, MAE:2.29103255) 
[343] EMBEDS(MSE:0.00708008, MAE:0.06689453)  LOGITS(MSE:10.28507042, MAE:2.51707506) 
[344] EMBEDS(MSE:0.00823975, MAE:0.07128906)  LOGITS(MSE:6.95430040, MAE:2.10027695) 
[345] EMBEDS(MSE:0.00759888, MAE:0.06787109)  LOGITS(MSE:8.59532070, MAE:2.34704590) 
[346] EMBEDS(MSE:0.00759888, MAE:0.06738281)  LOGITS(MSE:5.86769342, MAE:1.88388419) 
[347] EMBEDS(MSE:0.00509644, MAE:0.05615234)  LOGITS(MSE:7.78372574, MAE:2.19427156) 
[348] EMBEDS(MSE:0.00762939, MAE:0.06835938)  LOGITS(MSE:10.67490959, MAE:2.55733228) 
[349] EMBEDS(MSE:0.00686646, MAE:0.06494141)  LOGITS(MSE:7.96814013, MAE:2.21973133) 
[350] EMBEDS(MSE:0.00805664, MAE:0.07128906)  LOGITS(MSE:7.02027464, MAE:2.10348415) 
[351] EMBEDS(MSE:0.00698853, MAE:0.06591797)  LOGITS(MSE:4.96939707, MAE:1.76387382) 
[352] EMBEDS(MSE:0.00570679, MAE:0.05859375)  LOGITS(MSE:20.84104538, MAE:3.65089893) 
[353] EMBEDS(MSE:0.01025391, MAE:0.08007812)  LOGITS(MSE:8.83211231, MAE:2.33935738) 
[354] EMBEDS(MSE:0.00793457, MAE:0.07080078)  LOGITS(MSE:5.97983932, MAE:1.93569577) 
[355] EMBEDS(MSE:0.00665283, MAE:0.06445312)  LOGITS(MSE:6.21931791, MAE:1.97720289) 
[356] EMBEDS(MSE:0.00897217, MAE:0.07568359)  LOGITS(MSE:7.40648603, MAE:2.16352773) 
[357] EMBEDS(MSE:0.00836182, MAE:0.06787109)  LOGITS(MSE:11.96735096, MAE:2.75466561) 
[358] EMBEDS(MSE:0.00692749, MAE:0.06640625)  LOGITS(MSE:5.71109295, MAE:1.89420831) 
[359] EMBEDS(MSE:0.00598145, MAE:0.05883789)  LOGITS(MSE:6.88528872, MAE:2.07025695) 
[360] EMBEDS(MSE:0.00491333, MAE:0.04394531)  LOGITS(MSE:14.76066017, MAE:3.08753252) 
[361] EMBEDS(MSE:0.00717163, MAE:0.06298828)  LOGITS(MSE:11.24953556, MAE:2.68057823) 
[362] EMBEDS(MSE:0.00573730, MAE:0.04565430)  LOGITS(MSE:18.40716553, MAE:3.34388423) 
[363] EMBEDS(MSE:0.00738525, MAE:0.06787109)  LOGITS(MSE:5.06801367, MAE:1.77148020) 
[364] EMBEDS(MSE:0.00805664, MAE:0.07128906)  LOGITS(MSE:3.89302945, MAE:1.56474495) 
[365] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:3.54548049, MAE:1.48266649) 
[366] EMBEDS(MSE:0.00753784, MAE:0.06933594)  LOGITS(MSE:7.54265547, MAE:2.15067363) 
[367] EMBEDS(MSE:0.00891113, MAE:0.07421875)  LOGITS(MSE:6.10147095, MAE:1.94817770) 
[368] EMBEDS(MSE:0.00903320, MAE:0.07519531)  LOGITS(MSE:10.26166725, MAE:2.55781102) 
[369] EMBEDS(MSE:0.00714111, MAE:0.06396484)  LOGITS(MSE:5.43229389, MAE:1.82633257) 
[370] EMBEDS(MSE:0.00723267, MAE:0.06396484)  LOGITS(MSE:6.30494976, MAE:1.98554003) 
[371] EMBEDS(MSE:0.00738525, MAE:0.06152344)  LOGITS(MSE:7.32463551, MAE:2.13578010) 
[372] EMBEDS(MSE:0.00747681, MAE:0.06201172)  LOGITS(MSE:13.40330601, MAE:2.77225494) 
[373] EMBEDS(MSE:0.00830078, MAE:0.07128906)  LOGITS(MSE:13.86802959, MAE:2.96795726) 
[374] EMBEDS(MSE:0.01068115, MAE:0.07910156)  LOGITS(MSE:6.16395855, MAE:1.94624019) 
[375] EMBEDS(MSE:0.00643921, MAE:0.06127930)  LOGITS(MSE:13.69306946, MAE:2.99584222) 
[376] EMBEDS(MSE:0.00842285, MAE:0.07226562)  LOGITS(MSE:15.71619320, MAE:3.13294673) 
[377] EMBEDS(MSE:0.00866699, MAE:0.07421875)  LOGITS(MSE:11.91650867, MAE:2.72775126) 
[378] EMBEDS(MSE:0.01062012, MAE:0.07812500)  LOGITS(MSE:5.45510054, MAE:1.83613348) 
[379] EMBEDS(MSE:0.00521851, MAE:0.05566406)  LOGITS(MSE:13.33992386, MAE:2.95477104) 
[380] EMBEDS(MSE:0.00915527, MAE:0.07080078)  LOGITS(MSE:17.88440704, MAE:3.36257339) 
[381] EMBEDS(MSE:0.00720215, MAE:0.06738281)  LOGITS(MSE:11.38123512, MAE:2.69848514) 
[382] EMBEDS(MSE:0.00762939, MAE:0.07031250)  LOGITS(MSE:10.36286449, MAE:2.65123677) 
[383] EMBEDS(MSE:0.00787354, MAE:0.07031250)  LOGITS(MSE:7.56795359, MAE:2.18006063) 
[384] EMBEDS(MSE:0.00823975, MAE:0.06787109)  LOGITS(MSE:18.24985886, MAE:3.57088494) 
[385] EMBEDS(MSE:0.00729370, MAE:0.06787109)  LOGITS(MSE:3.90315843, MAE:1.56047297) 
[386] EMBEDS(MSE:0.00366211, MAE:0.03613281)  LOGITS(MSE:9.30034733, MAE:2.49851060) 
[387] EMBEDS(MSE:0.00854492, MAE:0.07275391)  LOGITS(MSE:10.28150368, MAE:2.54097271) 
[388] EMBEDS(MSE:0.00775146, MAE:0.06982422)  LOGITS(MSE:17.41518974, MAE:3.36204481) 
[389] EMBEDS(MSE:0.00823975, MAE:0.07128906)  LOGITS(MSE:4.97387743, MAE:1.73355484) 
[390] EMBEDS(MSE:0.00430298, MAE:0.04980469)  LOGITS(MSE:5.88794947, MAE:1.92578232) 
[391] EMBEDS(MSE:0.00454712, MAE:0.05273438)  LOGITS(MSE:14.64614964, MAE:3.17143130) 
[392] EMBEDS(MSE:0.00823975, MAE:0.07226562)  LOGITS(MSE:6.07619333, MAE:1.94074273) 
[393] EMBEDS(MSE:0.00640869, MAE:0.06347656)  LOGITS(MSE:6.78746128, MAE:2.02281141) 
[394] EMBEDS(MSE:0.00668335, MAE:0.06445312)  LOGITS(MSE:8.59806442, MAE:2.29766870) 
[395] EMBEDS(MSE:0.00686646, MAE:0.06494141)  LOGITS(MSE:7.36640787, MAE:2.12886310) 
[396] EMBEDS(MSE:0.00759888, MAE:0.06787109)  LOGITS(MSE:12.23171902, MAE:2.79784989) 
[397] EMBEDS(MSE:0.00695801, MAE:0.06591797)  LOGITS(MSE:3.64628601, MAE:1.47082591) 
[398] EMBEDS(MSE:0.00537109, MAE:0.05810547)  LOGITS(MSE:24.22287178, MAE:4.12700081) 
[399] EMBEDS(MSE:0.00769043, MAE:0.06933594)  LOGITS(MSE:5.56695700, MAE:1.86435127) 
[400] EMBEDS(MSE:0.00457764, MAE:0.05102539)  LOGITS(MSE:16.02422905, MAE:3.20923233) 
[401] EMBEDS(MSE:0.00741577, MAE:0.06591797)  LOGITS(MSE:12.29476929, MAE:2.80864382) 
[402] EMBEDS(MSE:0.00772095, MAE:0.06933594)  LOGITS(MSE:4.75626945, MAE:1.69360459) 
[403] EMBEDS(MSE:0.00561523, MAE:0.05883789)  LOGITS(MSE:10.85406399, MAE:2.63724089) 
[404] EMBEDS(MSE:0.00836182, MAE:0.07226562)  LOGITS(MSE:9.58081341, MAE:2.46423483) 
[405] EMBEDS(MSE:0.00939941, MAE:0.07714844)  LOGITS(MSE:11.00410938, MAE:2.65750790) 
[406] EMBEDS(MSE:0.00729370, MAE:0.06396484)  LOGITS(MSE:13.55376625, MAE:3.00053048) 
[407] EMBEDS(MSE:0.00683594, MAE:0.06494141)  LOGITS(MSE:4.53867340, MAE:1.68088698) 
[408] EMBEDS(MSE:0.00308228, MAE:0.03417969)  LOGITS(MSE:7.71277094, MAE:2.19772029) 
[409] EMBEDS(MSE:0.00521851, MAE:0.04077148)  LOGITS(MSE:16.30304337, MAE:3.21097922) 
[410] EMBEDS(MSE:0.00717163, MAE:0.06298828)  LOGITS(MSE:15.76030636, MAE:3.11240530) 
[411] EMBEDS(MSE:0.00701904, MAE:0.06250000)  LOGITS(MSE:6.76133537, MAE:2.05690122) 
[412] EMBEDS(MSE:0.00683594, MAE:0.06250000)  LOGITS(MSE:7.18978214, MAE:2.12371111) 
[413] EMBEDS(MSE:0.00817871, MAE:0.07177734)  LOGITS(MSE:6.91085339, MAE:2.11299181) 
[414] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:5.91499281, MAE:1.93977666) 
[415] EMBEDS(MSE:0.00878906, MAE:0.07275391)  LOGITS(MSE:14.06545830, MAE:2.91530490) 
[416] EMBEDS(MSE:0.01092529, MAE:0.08251953)  LOGITS(MSE:7.33107376, MAE:2.14834571) 
[417] EMBEDS(MSE:0.00741577, MAE:0.06591797)  LOGITS(MSE:8.46459007, MAE:2.30561256) 
[418] EMBEDS(MSE:0.00903320, MAE:0.07031250)  LOGITS(MSE:16.31793976, MAE:3.21823716) 
[419] EMBEDS(MSE:0.00842285, MAE:0.06835938)  LOGITS(MSE:14.07595348, MAE:2.94222426) 
[420] EMBEDS(MSE:0.00994873, MAE:0.07470703)  LOGITS(MSE:13.03323174, MAE:2.80235171) 
[421] EMBEDS(MSE:0.00692749, MAE:0.05981445)  LOGITS(MSE:8.04041767, MAE:2.26967382) 
[422] EMBEDS(MSE:0.00521851, MAE:0.04077148)  LOGITS(MSE:19.37577629, MAE:3.56517267) 
[423] EMBEDS(MSE:0.00717163, MAE:0.06298828)  LOGITS(MSE:18.98443985, MAE:3.36922145) 
[424] EMBEDS(MSE:0.00756836, MAE:0.06640625)  LOGITS(MSE:22.06432724, MAE:3.74884415) 
[425] EMBEDS(MSE:0.01068115, MAE:0.08203125)  LOGITS(MSE:15.17683125, MAE:3.08222389) 
[426] EMBEDS(MSE:0.01055908, MAE:0.08203125)  LOGITS(MSE:6.05512047, MAE:1.94554806) 
[427] EMBEDS(MSE:0.00711060, MAE:0.06689453)  LOGITS(MSE:11.16531467, MAE:2.72769046) 
[428] EMBEDS(MSE:0.00549316, MAE:0.05053711)  LOGITS(MSE:18.14039040, MAE:3.39463115) 
[429] EMBEDS(MSE:0.00506592, MAE:0.05151367)  LOGITS(MSE:6.49716568, MAE:2.01583171) 
[430] EMBEDS(MSE:0.00842285, MAE:0.07324219)  LOGITS(MSE:6.09004736, MAE:1.95169675) 
[431] EMBEDS(MSE:0.00933838, MAE:0.07617188)  LOGITS(MSE:6.88236904, MAE:2.05771685) 
[432] EMBEDS(MSE:0.00753784, MAE:0.06884766)  LOGITS(MSE:5.06693983, MAE:1.78487372) 
[433] EMBEDS(MSE:0.00619507, MAE:0.06005859)  LOGITS(MSE:8.57096291, MAE:2.32464075) 
[434] EMBEDS(MSE:0.00726318, MAE:0.06347656)  LOGITS(MSE:6.01455212, MAE:1.95060837) 
[435] EMBEDS(MSE:0.00668335, MAE:0.06225586)  LOGITS(MSE:9.56546307, MAE:2.46312332) 
[436] EMBEDS(MSE:0.00775146, MAE:0.07080078)  LOGITS(MSE:5.65562248, MAE:1.88242197) 
[437] EMBEDS(MSE:0.00622559, MAE:0.05957031)  LOGITS(MSE:8.85104561, MAE:2.35629439) 
[438] EMBEDS(MSE:0.00830078, MAE:0.06933594)  LOGITS(MSE:17.62204742, MAE:3.40822387) 
[439] EMBEDS(MSE:0.00701904, MAE:0.06445312)  LOGITS(MSE:17.35676765, MAE:3.34557295) 
[440] EMBEDS(MSE:0.00805664, MAE:0.07177734)  LOGITS(MSE:8.89520741, MAE:2.35513520) 
[441] EMBEDS(MSE:0.00512695, MAE:0.05517578)  LOGITS(MSE:14.44553185, MAE:3.14265084) 
[442] EMBEDS(MSE:0.00704956, MAE:0.06787109)  LOGITS(MSE:8.53143692, MAE:2.32212639) 
[443] EMBEDS(MSE:0.00358582, MAE:0.04467773)  LOGITS(MSE:20.86907196, MAE:3.75507450) 
[444] EMBEDS(MSE:0.00772095, MAE:0.06933594)  LOGITS(MSE:8.90988636, MAE:2.33172083) 
[445] EMBEDS(MSE:0.00717163, MAE:0.06640625)  LOGITS(MSE:11.08434010, MAE:2.67253017) 
[446] EMBEDS(MSE:0.00494385, MAE:0.05126953)  LOGITS(MSE:11.83797646, MAE:2.72048306) 
[447] EMBEDS(MSE:0.00787354, MAE:0.07128906)  LOGITS(MSE:8.15660763, MAE:2.24384117) 
[448] EMBEDS(MSE:0.00625610, MAE:0.06030273)  LOGITS(MSE:13.35486507, MAE:2.86633730) 
[449] EMBEDS(MSE:0.00964355, MAE:0.07470703)  LOGITS(MSE:10.26062489, MAE:2.55116868) 
[450] EMBEDS(MSE:0.00817871, MAE:0.06640625)  LOGITS(MSE:15.32398796, MAE:3.10328245) 
[451] EMBEDS(MSE:0.00793457, MAE:0.06982422)  LOGITS(MSE:10.05740738, MAE:2.57022357) 
[452] EMBEDS(MSE:0.00000000, MAE:0.00000000)  LOGITS(MSE:10.02477741, MAE:2.67087030) 
[453] EMBEDS(MSE:0.00399780, MAE:0.04565430)  LOGITS(MSE:17.98828506, MAE:3.46209049) 
[454] EMBEDS(MSE:0.00604248, MAE:0.06201172)  LOGITS(MSE:9.29011345, MAE:2.44175625) 
[455] EMBEDS(MSE:0.00427246, MAE:0.04492188)  LOGITS(MSE:8.43882656, MAE:2.28919768) 
[456] EMBEDS(MSE:0.00885010, MAE:0.07519531)  LOGITS(MSE:12.79547596, MAE:2.81852770) 
[457] EMBEDS(MSE:0.00891113, MAE:0.07519531)  LOGITS(MSE:9.21367264, MAE:2.41299200) 
[458] EMBEDS(MSE:0.00488281, MAE:0.04858398)  LOGITS(MSE:13.51471996, MAE:2.92073607) 
"""